In [1]:
import pandas as pd

In [3]:
mohdata = pd.read_csv("https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_state.csv")
print(mohdata.columns)
mohdata.head()

Index(['date', 'state', 'cases_new'], dtype='object')


,date,state,cases_new
0,2020-03-16,Johor,52
1,2020-03-16,Kedah,31
2,2020-03-16,Kelantan,18
3,2020-03-16,Melaka,14
4,2020-03-16,Negeri Sembilan,42


In [6]:
googledata = pd.read_csv("google-mobility-data-malaysia.csv")
googledata['state'].unique()

array(['Kuala Lumpur', 'Johor', 'Kedah', 'Kelantan', 'Labuan', 'Melaka',
       'Negeri Sembilan', 'Pahang', 'Penang', 'Perak', 'Perlis',
       'Putrajaya', 'Sabah', 'Sarawak', 'Selangor', 'Terengganu'],
      dtype=object)

In [9]:
print(mohdata['state'].unique())
mohdata['state'].replace({"W.P. ":""}, regex=True, inplace=True)
mohdata.loc[mohdata['state']=='Pulau Pinang', 'state'] = 'Penang'
print(mohdata.state.unique())
mohdata.describe(include="all")

['Johor' 'Kedah' 'Kelantan' 'Melaka' 'Negeri Sembilan' 'Pahang' 'Perak'
 'Perlis' 'Penang' 'Sabah' 'Sarawak' 'Selangor' 'Terengganu'
 'Kuala Lumpur' 'Labuan' 'Putrajaya']
['Johor' 'Kedah' 'Kelantan' 'Melaka' 'Negeri Sembilan' 'Pahang' 'Perak'
 'Perlis' 'Penang' 'Sabah' 'Sarawak' 'Selangor' 'Terengganu'
 'Kuala Lumpur' 'Labuan' 'Putrajaya']


,date,state,cases_new
count,8176,8176,8176.000000
unique,511,16,NaN
top,2021-01-30,Sabah,NaN
freq,16,511,NaN
mean,NaN,NaN,154.420254
std,NaN,NaN,490.243637
min,NaN,NaN,-101.000000
25%,NaN,NaN,0.000000
50%,NaN,NaN,9.000000
75%,NaN,NaN,118.000000


In [10]:
mohdata.to_csv("../data/moh-covid-cases.csv", index=False)